In [ ]:
# Install necessary libraries
!pip install transformers spacy gensim datasets openai gradio nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found

In [ ]:
# Import libraries
import spacy
from nltk.corpus import wordnet
import openai
import gradio as gr
from datasets import load_dataset

In [ ]:
!pip install gradio

In [ ]:
# Set up OpenAI API key
openai.api_key = "sk-your-api-key"  # Replace with your actual API key

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
!pip install nltk
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
!pip install openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [ ]:
# #new version

import openai
import gradio as gr
from nltk.corpus import wordnet

# Function to validate suggestions using multiple LLMs
def validate_with_multiple_models(text, suggestion, max_validations=3):
    models = ["gpt-4", "gpt-3.5-turbo"]
    validated_suggestions = []

    # Truncate suggestions to avoid redundant validations
    truncated_suggestions = suggestion.split("\n")[:max_validations]

    for model in models:
        for partial_suggestion in truncated_suggestions:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are an expert in validating vocabulary suggestions."},
                    {"role": "user", "content": f"Validate if this suggestion '{partial_suggestion}' fits appropriately in the context of the text: '{text}'"}
                ],
                max_tokens=150,
                temperature=0.7
            )
            validated_suggestions.append(response['choices'][0]['message']['content'].strip())
    return validated_suggestions

# Function to determine if the sentence is already good
def is_sentence_good(text, proficiency_level="beginner"):
    instruction = f"Evaluate the following sentence and determine if it is appropriate and well-written for a {proficiency_level} language learner: '{text}'. Respond with 'Good' if the sentence is appropriate or suggest improvements if necessary."

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant evaluating sentence quality based on proficiency level."},
            {"role": "user", "content": instruction}
        ],
        max_tokens=150,
        temperature=0.7
    )
    evaluation = response['choices'][0]['message']['content'].strip()

    if "Good" in evaluation:
        return True, f"Your sentence is really good for your {proficiency_level} level."
    else:
        return False, evaluation

# Function to get GPT-4 suggestions based on proficiency level (for Output 1)
def get_gpt4_suggestions(text, proficiency_level="beginner"):
    if proficiency_level == "beginner":
        instruction = "Suggest simple vocabulary words that are easy for a beginner language learner to understand and improve this sentence."
    elif proficiency_level == "intermediate":
        instruction = "Suggest intermediate-level vocabulary words that provide more nuanced meanings but are still relatively easy for language learners."
    elif proficiency_level == "advanced":
        instruction = "Suggest advanced-level vocabulary words that are sophisticated and add more complexity and depth to the sentence."

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that suggests vocabulary based on language proficiency."},
            {"role": "user", "content": f"{instruction} Here is the sentence: {text}"}
        ],
        max_tokens=150,
        temperature=0.7
    )
    suggestions = response['choices'][0]['message']['content'].strip()
    return suggestions

# Function to get better word equivalents (Output 2)
def get_better_equivalents(text):
    words = text.split()  # Split the input text into words
    word_suggestions = []
    index = 1  # Used to number the output suggestions

    for word in words:
        synonyms = set()
        # Get synonyms from WordNet for each word
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())

        # Filter out identical words and irrelevant synonyms, keeping only relevant ones
        relevant_synonyms = {syn for syn in synonyms if syn.lower() != word.lower() and len(syn) > 2}

        # If synonyms are found, check their context
        if relevant_synonyms:
            # Ask GPT-4 to filter synonyms based on context
            filtered_synonyms = filter_synonyms_by_context(word, list(relevant_synonyms), text)
            if filtered_synonyms:
                # Format the output as '1. word == synonym1, synonym2'
                word_suggestions.append(f"{index}. {word.capitalize()} == {', '.join(filtered_synonyms[:5])}")  # Show up to 5 synonyms
                index += 1

    return "\n".join(word_suggestions)

# Function to filter synonyms based on context using GPT-4 (for Output 2)
def filter_synonyms_by_context(word, synonyms, sentence):
    prompt = f"Here is a sentence: '{sentence}'. Which of these words can replace '{word}' in the sentence without changing the meaning: {', '.join(synonyms)}?"

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert in contextual word replacement."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )

    # Extract GPT-4's response and clean the result to show just the synonyms without extra text
    filtered_response = response['choices'][0]['message']['content'].strip()

    # Clean the response to avoid verbose explanations and keep only useful synonyms
    filtered_synonyms = [syn.strip() for syn in filtered_response.split(",") if len(syn.strip()) > 0]

    return filtered_synonyms

# Unified function for Gradio Interface
def vocabulary_and_word_equivalents_tool(text, proficiency):
    # Check if the sentence is already good
    is_good, feedback_or_suggestions = is_sentence_good(text, proficiency)

    if is_good:
        return text, feedback_or_suggestions, "", ""
    else:
        # Get vocabulary suggestions for Output 1
        vocab_suggestions = get_gpt4_suggestions(text, proficiency)
        # Get word equivalents for Output 2
        better_words = get_better_equivalents(text)
        # Validate suggestions using multiple models
        validated = validate_with_multiple_models(text, vocab_suggestions)
        return text, vocab_suggestions, better_words, validated

# Gradio Interface setup
iface = gr.Interface(
    fn=vocabulary_and_word_equivalents_tool,  # Single function to handle all tasks
    inputs=[
        gr.Textbox(lines=3, placeholder="Enter your sentence here..."),
        gr.Radio(["beginner", "intermediate", "advanced"], label="Proficiency Level")
    ],
    outputs=[
        gr.Textbox(label="Original Text"),  # Original text input displayed
        gr.Textbox(label="Feedback or Suggestions"),  # Feedback if sentence is good
        gr.Textbox(label="Suggested Text Based on Proficiency Level"),  # Output 1
        gr.Textbox(label="Better Equivalent Words"),  # Output 2
    ],
    title="Vocabulary Expansion Tool",
    description="Expand your vocabulary with context-aware suggestions. If your sentence is good for the selected level, you'll get confirmation instead of new suggestions."
)

# Launch the Gradio interface
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a552f8daaa605c825.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
